In [ ]:
!pip install fasttext

     |████████████████████████████████| 71kB 4.7MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3042911 sha256=fc4fc7bd58f15838ee8eba57744313636af6507c9a47376d83d10115fba4b1b1
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
# baseline과 fastetext공유글을 참고하여 진행

In [ ]:
import fasttext
import pandas as pd
import numpy as np
import re

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
train_path = '/content/gdrive/MyDrive/작가분류/open/train.csv'
test_path = '/content/gdrive/MyDrive/작가분류/open/test_x.csv'
submission_path = '/content/gdrive/MyDrive/작가분류/open/sample_submission.csv'

In [ ]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
submission = pd.read_csv(submission_path, index_col=False)

In [ ]:
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [ ]:
# 부호제거

In [ ]:
def clean_sentenct(text):
  return re.sub(r'[^A-Za-z0-9 ]', '', text)

In [ ]:
# 불용어 제거
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [ ]:
def clean_stopword(text):
  final_text = []
  for i in text.split():
    if i.strip().lower() not in stopwords:
      final_text.append(i.strip())
  return " ".join(final_text)

In [ ]:
# 부호, 불용어 제거
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(clean_sentenct)
test['text'] = test['text'].apply(clean_sentenct)

In [ ]:
train

,index,text,author
0,0,he was almost choking there was so much so muc...,3
1,1,your sister asked for it i suppose,2
2,2,she was engaged one day as she walked in peru...,1
3,3,the captain was in the porch keeping himself c...,4
4,4,have mercy gentlemen odin flung up his hands d...,3
...,...,...,...
54874,54874,is that you mr smith odin whispered i hardly d...,2
54875,54875,i told my plan to the captain and between us w...,4
54876,54876,your sincere wellwisher friend and sister luc...,1
54877,54877,then you wanted me to lend you money,3


In [ ]:
# 전처리를 진행하지 않고 하는것이 더 잘 나옴

In [ ]:
file = open('fasttexttrain.txt','w+')
for i in train.index:
    line = '__label__' + str(train['author'][i])+' '+train['text'][i]
    file.write(line+"\n")

In [ ]:
text_clf_model = fasttext.train_supervised('fasttexttrain.txt', epoch=50, minCount=2, maxn=10, verbose=0,wordNgrams=2)

In [ ]:
reuslt = text_clf_model.predict("He was almost choking. There was so much, so much he wanted to say, but strange exclamations were all that came from his lips. The Pole gazed fixedly at him, at the bundle of notes in his hand; looked at odin, and was in evident perplexity.", k=5)
print(reuslt)

(('__label__3', '__label__0', '__label__2', '__label__1', '__label__4'), array([9.99771774e-01, 1.53049667e-04, 7.23329213e-05, 3.39585131e-05,
       1.89975308e-05]))


In [ ]:
for i in test.index:
    lable, proba = text_clf_model.predict(test['text'][i], k=5)
    for la, pr in zip(lable, proba):
        if '__label__0' == la:
            submission.loc[i, '0'] = pr
        elif '__label__1' == la:
            submission.loc[i, '1'] = pr
        elif '__label__2' == la:
            submission.loc[i, '2'] = pr
        elif '__label__3' == la:
            submission.loc[i, '3'] = pr
        elif '__label__4' == la:
            submission.loc[i, '4'] = pr

submission.to_csv('/content/gdrive/MyDrive/작가분류/result10_fasttext.csv', index=False)
print('end')

end
